In [ ]:
# default_exp database_proxy
%load_ext autoreload
%autoreload 2

# Database Proxy
> Accesses the database

In [ ]:
#export
import pandas as pd
import sqlite3
import time
from kirby.properties import properties

Tables organization: 
Entities_[letter or not_alpha]
entity_id: Primary key, Unique 
label: name of the entity
description: Short description 

Properties_relations Table
entity_id: relationship of  
relations: List of all relations

Aliases Table
entity_id 
alias: Other labels. 

Properties Table
property_id Unique primary key  
label:Description of the Property 

In [ ]:
#export
class WikiDatabase:
    conn = None
    def __init__(self):
        try:
            self.properties_dict = properties()
            self.conn = sqlite3.connect("/data/db/wiki_data.db")
            print(self.conn)
        except Exception as e:
            print(e)
            exit(-1)

    def exit_procedure(self):
        pass
#         self.conn.close()
#         exit(-1)
        

    @staticmethod
    def get_table_name(entity_label):
        """
        @:param self:
        @:param label of the entity for which you need to
        :rtype: string
        @:return name of the table where entity is stored
        """
        if entity_label[0].isalpha():
            return entity_label[0].lower()
        else:
            return 'not_alpha'

    @staticmethod
    def clean_relations(relations_string):
        """
        :param relations_string: List of all the relations returned by database
        :type relations_string: string
        :return: list of all relations
        :rtype: List
        """
        relations = []
        relation_row = []
        # print(relations_string)
        for idx, relation in enumerate(relations_string.split(',')):
            if idx & 1 == 1:
                relation_row.append(relation[1:-1])
            else:
                relation_row.append(relation[1:])
            if len(relation_row) == 2:
                relations.append(relation_row + [])
                # print(relation_row)
                relation_row.clear()
        # print(relations)
        return relations

    @staticmethod
    def get_property_label(relations_id):
        """
            :param property_id: id of the property to look for
            :type property_id:
            :return: label of the given id, None if the id does not exist
            :rtype:
        """
        return property_dict[relations_id]

    @staticmethod
    def remove_quotations(label):
        index = 0
        while index < len(label):
            if label[index] == '\'':
                label = label[:index] + '\'' + label[index:]
                index += 1
            elif label[index] == "\"":
                label = label[:index - 1] + '\'' + '\'' + label[index:]
                index += 1
            index += 1

        return label

    # TODO: Test the return value
    # TODO: change to check in the entities table
    def get_similar_entities_label(self, label):
        """
        Returns a list of entities id where the label matches the entity_label name
        :param self:
        :type self:
        :param label:
        :type label:
        :return: list of all the entities id and label where the label matched
        :rtype: Array containing all similar entities
        """
        entities_id = None
        try:
            print('I love me!')
            entities_id = pd.read_sql_query(
                "SELECT * FROM Entities WHERE label  LIKE \"%{}%\";".format(self.remove_quotations(label)),
                self.conn)
        except Exception as e:
            print(e)
            self.exit_procedure()
        return entities_id.values.tolist()

    def get_entities_by_label_extensive(self, label):
        """
        Returns a list of entities id where the label matches the entity_label name
        :param self:
        :type self:
        :param label:
        :type label:
        :return: list of all the entities id and label where the label matched
        :rtype: Array containing all similar entities
        """
        table_name = self.get_table_name(label)
        entities_id = None
        try:
            entities_id = pd.read_sql_query(
                "SELECT * FROM Entities_{} WHERE label  LIKE \"{}%\";"\
                .format(table_name, self.remove_quotations(label)), self.conn)
            return entities_id.values.tolist()
        except Exception as e:
            print(e)
            self.exit_procedure()
        

    def get_entity_by_label(self, label):
        """
        Search for the id, label and description of an specific identity
        :param label: string of the label to look for
        :type label: string
        :return: id, label and description of the label
        :rtype: pandas.core.series.Series
        """
        entity_id = None
        table_name = self.get_table_name(label)
        try:
            entity_id = pd.read_sql_query(
                "SELECT * FROM Entities_{} WHERE label =  \"{}\";"\
                .format(table_name, self.remove_quotations(label)), self.conn)
            entities = entity_id.values.tolist()
            # Return the first entity
            entities = self._sort_entities(entities)
            return entities[0]
        except Exception as e:
            print(e)
            self.exit_procedure()
            
    def _sort_entities(self, entities):
        """
            Given a list of entities, sort the list where the lowest id number is first.
            
            We are assuming that the most likely entity is the one with the lowest id number
            in the knowledge base.
        """
        # Chops off the first letter Q and casts the string as a number
        return sorted(entities, key=lambda x:int(x[0][1:]))
        
        
        
    def get_entity_associations(self, entity_id):
        """
        Search for all properties of the entity
        Return all the
        :param entity_id: id of a entity
        :type entity_id: str
        :return: data frame with a string of the relations
        :rtype: pandas.core.series.Series
        """
        entity_properties = None
        try:
            entity_properties = pd.read_sql_query(
                "SELECT * FROM Properties_relations WHERE entity_id = \"{}\";".format(entity_id),
                self.conn)
        except Exception as e:
            print(e)
            self.exit_procedure()
        if entity_properties.empty:
            return None
        return self.clean_relations(entity_properties['relations'].values[0])

    def get_entity_by_id(self, entity_id):

        relation_label = None
        try:
            relation_label = pd.read_sql_query(
                "SELECT label, description FROM Entities WHERE entity_id = \"{}\"".format(entity_id),
                self.conn)
        except Exception as e:
            print(e)
            self.exit_procedure()
        if relation_label.empty:
            return None
        return relation_label.iloc[0]["label"]

    def get_property_string(self, property_id, related_entity_id):
        """
        :param property_id:
        :type property_id: string
        :param related_entity_id:
        :type related_entity_id: string
        :return:
        :rtype: string describing the relationship
        """
        entity_label = None
        related_entity_label = None
        property_name = None
        try:
            # entity_label = get_entity_label(entity_id)
            related_entity_label = self.get_entity_by_id(related_entity_id)
            property_name = self.properties_dict[property_id]
            # print(related_entity_label, property_name)
        except Exception as e:
            print(e)
            self.exit_procedure()
        return property_name, related_entity_label

# Testing

In [ ]:
# Creation
db = WikiDatabase()
assert isinstance(db, WikiDatabase)

In [ ]:
# Test Connection
assert db.conn is not None, "No DB connection"

In [ ]:
# Test sort
entities = [['Q4909106', 'Bill Gates', 'American frontiersman'], ['Q1181671', 'Bill Gates', 'Wikimedia disambiguation page'], ['Q5284', 'Bill Gates', 'American business magnate and philanthropist']]
entities = db._sort_entities(entities)
assert entities == [['Q5284', 'Bill Gates', 'American business magnate and philanthropist'], ['Q1181671', 'Bill Gates', 'Wikimedia disambiguation page'], ['Q4909106', 'Bill Gates', 'American frontiersman']],\
'Errorin `_sort_entities` function in `database_proxy` class'
print(entities)

[['Q5284', 'Bill Gates', 'American business magnate and philanthropist'], ['Q1181671', 'Bill Gates', 'Wikimedia disambiguation page'], ['Q4909106', 'Bill Gates', 'American frontiersman']]


In [ ]:
wiki = WikiDatabase()
result = wiki.get_entity_by_label('Bill Gates')
# assert result == [['Q4909106', 'Bill Gates', 'American frontiersman'], ['Q1181671', 'Bill Gates', 'Wikimedia disambiguation page'], ['Q5284', 'Bill Gates', 'American business magnate and philanthropist']], 'Query failed'
print(result)

['Q5284', 'Bill Gates', 'American business magnate and philanthropist']


In [ ]:
properties = wiki.get_entity_associations(result[0])
assert properties ==  [['P25', 'Q454928'], ['P19', 'Q5083'], ['P26', 'Q463877'], ['P106', 'Q131524'], ['P27', 'Q30'], ['P108', 'Q655286'], ['P69', 'Q7879362'], ['P910', 'Q7112419'], ['P31', 'Q5'], ['P551', 'Q1506847'], ['P166', 'Q12201445'], ['P1343', 'Q17311605'], ['P735', 'Q12344159'], ['P39', 'Q484876'], ['P734', 'Q16870134'], ['P40', 'Q23011254'], ['P172', 'Q7435494'], ['P463', 'Q463303'], ['P3438', 'Q575937'], ['P1412', 'Q1860'], ['P21', 'Q6581097'], ['P1830', 'Q2283'], ['P1441', 'Q12125162'], ['P6886', 'Q1860'], ['P641', 'Q188966'], ['P552', 'Q789447'], ['P22', 'Q684014'], ['P8017', 'L19333-F1'], ['P6553', 'L485'], ['P3373', 'Q92466067'], ['P1344', 'Q16972891']], 'Properties failed'
for property_r in wiki.get_entity_associations('Q5284'): 
    print(wiki.get_property_string(property_r[0], property_r[1]))


('mother', 'Mary Maxwell Gates')
('place of birth', 'Seattle')
('spouse', 'Melinda Gates')
('occupation', 'entrepreneur')
('country of citizenship', 'United States of America')
('employer', 'Bill & Melinda Gates Foundation')
('educated at', 'Lakeside School')
("topic's main category", 'Category:Bill Gates')
('instance of', 'human')
('residence', 'Medina')
('award received', 'Knight Commander of the Order of the British Empire')
('described by source', 'Lentapedia (full versions)')
('given name', 'William')
('position held', 'chief executive officer')
('family name', 'Gates')
('child', 'Jennifer Katherine Gates')
('ethnic group', 'Scotch-Irish Americans')
('member of', 'American Academy of Arts and Sciences')
('vehicle normally used', 'Porsche 959')
('languages spoken, written or signed', 'English')
('sex or gender', 'male')
('owner of', 'Microsoft')
('present in work', 'iSteve')
('writing language', 'English')
('sport', 'contract bridge')
('handedness', 'left-handedness')
('father', 'W